In [2]:
#!pip install bertopic
#!pip install openai

In [3]:
import pandas as pd
import time
from matplotlib import pyplot as plt
from datetime import datetime
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import re

In [4]:
def save(df):
    ts = int(time.time())
    df.to_excel("./files/{}.xlsx".format(ts))
    print(str(ts)+".xlsx has been saved")

In [5]:
path = "https://raw.githubusercontent.com/ayamlearning/masters_thesis_model_final/main/data/sample.csv"
df = pd.read_csv(path,delimiter=",",dtype=str)
print(df.shape)

(21098, 33)


In [6]:
df.columns

Index(['id', 'tweet_id', 'username', 'followers_count', 'user_verified',
       'text', 'created_at', 'like_count', 'retweet_count', 'reply_count',
       'location', 'lang', 'generation', 'ass_hashtags', 'keyword_exists',
       'prep_text', 'year', 'month', 'day', 'week', 'time', 'hour', 'minute',
       'vader_polarity', 'bert_polarity', 'date', 'city', 'district',
       'province', 'country', 'quater', 'weekday', 'tweet_lang'],
      dtype='object')

In [7]:
df.sample(5)

,id,tweet_id,username,followers_count,user_verified,text,created_at,like_count,retweet_count,reply_count,...,vader_polarity,bert_polarity,date,city,district,province,country,quater,weekday,tweet_lang
3334,50482,1326961275126099975,Malama_Mukonde,7080.0,False,Step 1 completed... nomba apa kwashala Voters ...,2020-11-12 20:53:16+02,2.0,0.0,0.0,...,Neutral,Neutral,2020-11-12 20:53:16+02:00,NaN,NaN,NaN,NaN,4,Thursday,en
1389,14937,1395027720573227011,AlumbweZuchi,2241.0,False,As a musician singing a song for a political p...,2021-05-19 16:45:01+02,0.0,0.0,0.0,...,Positive,Negative,2021-05-19 16:45:01+02:00,Ndola,Ndola District,Copperbelt Province,Zambia,2,Wednesday,en
3149,43563,1340226861046095872,Patlups,1356.0,False,Welcome to Western province. Your working gove...,2020-12-19 11:25:58+02,0.0,0.0,0.0,...,Neutral,Positive,2020-12-19 11:25:58+02:00,NaN,NaN,NaN,NaN,4,Saturday,en
19426,71362,1029966630867148800,ZambiaElections,37562.0,False,LEGAL FRAMEWORK FOR THE CONDUCT OF ELECTIONS I...,2018-08-16 07:42:35+02:00,1.0,0.0,0.0,...,Neutral,Neutral,2018-08-16 07:42:35+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,3,Thursday,en
9230,49146,1126707874833489920,GoodwillButhel,2350.0,False,@ThokozaniShon13 @AdvBarryRoux Remember this i...,2019-05-10 06:37:45+02:00,0.0,0.0,0.0,...,Negative,Negative,2019-05-10 06:37:45+02:00,NaN,NaN,NaN,NaN,2,Friday,en


#Neutral Sentiments

In [8]:
query = df.query("vader_polarity == '{}'".format("Neutral"))
query.sample(5)

,id,tweet_id,username,followers_count,user_verified,text,created_at,like_count,retweet_count,reply_count,...,vader_polarity,bert_polarity,date,city,district,province,country,quater,weekday,tweet_lang
15246,66911,1308365926283325440,ZambiaElections,37562.0,False,Press Release: Clarification on eligibility to...,2020-09-22 13:21:59+02:00,8.0,8.0,6.0,...,Neutral,Neutral,2020-09-22 13:21:59+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,3,Tuesday,en
11092,62466,1413461696275001347,2021Zambian,2714.0,False,"PF Media Director Antonio Mwanza, who once vow...",2021-07-09 13:35:03+02:00,0.0,0.0,0.0,...,Neutral,Negative,2021-07-09 13:35:03+02:00,NaN,NaN,NaN,NaN,3,Friday,en
1991,17728,1418487324619460608,KhozhiJ,1731.0,False,12 DAYS TO GO.. AND YOU KNOW WHAT THAT MEANS R...,2021-07-23 10:25:07+02,2.0,0.0,0.0,...,Neutral,Neutral,2021-07-23 10:25:07+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,3,Friday,en
7994,21004,801744891109928960,AfricasMartha,11959.0,False,"The case of #Zambia, the election aftermath!!!...",2016-11-24 13:10:52+02:00,0.0,1.0,0.0,...,Neutral,Neutral,2016-11-24 13:10:52+02:00,NaN,NaN,NaN,Zambia,4,Thursday,en
13957,65498,1334055025006555139,queeningwitgifs,322.0,False,"Trade unions are fragmented because of ""brown ...",2020-12-02 10:41:18+02:00,2.0,0.0,0.0,...,Neutral,Negative,2020-12-02 10:41:18+02:00,London Borough of Sutton,Greater London,England,United Kingdom,4,Wednesday,en


In [9]:
query.text = query.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
query.text = query.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
query.text = query.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
timestamps = query.date.to_list()
tweets = query.text.to_list()

In [10]:
docs = list(map(str,  query['prep_text']))
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/406 [00:00<?, ?it/s]

2023-10-21 06:19:51,562 - BERTopic - Transformed documents to Embeddings
2023-10-21 06:20:36,661 - BERTopic - Reduced dimensionality
2023-10-21 06:21:25,302 - BERTopic - Clustered reduced embeddings


In [11]:
topic_model.visualize_barchart()

In [12]:
topics_over_time = topic_model.topics_over_time(tweets, timestamps, nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)

20it [00:04,  4.13it/s]


In [13]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

In [14]:
topic_model.visualize_hierarchy(top_n_topics=7)

In [15]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])
df

100%|██████████| 13/13 [00:03<00:00,  3.64it/s]


,day,left,remember,keep,best,attitude,avoid,getting,conflict,law,vote,count,visiting
4_day_remaining_till_left,0.201,0.310,0.310,0.310,0.110,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8_calendar_general_checklist_2021,0.105,0.105,0.105,0.105,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
40_saved_closer_keep_working,0.155,0.271,0.271,0.271,0.116,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
100_law_lawyer_lawlessness_studying,0.000,0.000,0.000,0.000,0.000,0.000,0.110,0.240,0.373,0.500,0.391,0.261,0.127
115_voice_alot_deflected_talked,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.160,0.160,0.160,0.160
121_day_13hrs_snub_sevendays,0.154,0.154,0.154,0.154,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
142_wire_ku_ka_left,0.163,0.286,0.286,0.286,0.123,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
214_booth_remember_smile_shot,0.245,0.424,0.561,0.561,0.317,0.137,0.000,0.000,0.000,0.000,0.000,0.000,0.000
229_internet_keep_switching_elearning,0.262,0.459,0.610,0.749,0.487,0.290,0.139,0.000,0.000,0.000,0.000,0.000,0.000


In [16]:
topic_model.get_topic(0)

[('zambia', 0.012319024298632915),
 ('election', 0.011061801583790784),
 ('year', 0.009518793688192842),
 ('month', 0.008195022849522652),
 ('next', 0.008123187715081793),
 ('contested', 0.007569538801852248),
 ('fragmented', 0.006705398196357326),
 ('turmoil', 0.006705398196357326),
 ('lusaka', 0.006618102786322268),
 ('towards', 0.005772075781611553)]

In [17]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)